# Amazon Bedrock Agent -Code Interpreter Example

In [3]:
import boto3
import json
import time, random 
import uuid, string
import io

### Pick the foundation model as of today 08/01/2024 Sonnet and Haiku are supported

In [4]:
region_name = 'us-east-1'

# The foundation model to use (for 'code interpreter' it must either Sonnet or Haiku).
foundationModel = 'anthropic.claude-3-sonnet-20240229-v1:0'

In [5]:
randomSuffix = "".join(
    random.choices(string.ascii_uppercase + string.digits, k=5)
)

### Create the IAM policy

In [6]:
# Load IAM policies from JSON files
with open('trust_policy.json') as f:
    trust_policy = json.load(f)

with open('bedrock_Agent_policy.json') as f:
    policy = json.load(f)

# Initialize IAM client
iam = boto3.client('iam')

# Generate a random suffix for unique naming
random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=5))
role_name = f"test-agent-{random_suffix}"

print("Creating the IAM policy and role...")

# Create IAM role and attach policy
role = iam.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument=json.dumps(trust_policy)
)
iam.put_role_policy(
    RoleName=role_name,
    PolicyName=f"policy-test-agent1-{random_suffix}",
    PolicyDocument=json.dumps(policy)
)

roleArn = role['Role']['Arn']
print(f"IAM Role: {roleArn[:13]}{'*' * 12}{roleArn[25:]}")

Creating the IAM policy and role...
IAM Role: arn:aws:iam::************:role/test-agent-FYS00


###  Set up Bedrock Agent and IAM clients

In [7]:

bedrock_agent = boto3.client(service_name = 'bedrock-agent', region_name = region_name)


agentName = 'code-interpreter-test-agent1'

### Defining the Bedrock Agent PROMPT

In [8]:


# Define the agent's personality and behavior
instruction = """You are an AI agent with features for code execution, code generation, and data analysis. Your primary function is to assist users by solving problems and fulfilling requests through these capabilities. Here are your key attributes and instructions:

Code Execution
Python Environment: You have access to a Python environment where you can write and execute code in real-time.
Accuracy: When asked to perform calculations or data manipulations, always use this code execution capability to ensure accuracy.
Reporting: After executing code, report the exact output and explain the results.
Data Analysis
Complex Data Analysis: You excel at complex data analysis tasks. This includes statistical analysis and machine learning applications.
Systematic Approach: Approach data analysis tasks systematically: understand the problem, prepare the data, perform the analysis, and interpret the results.
Problem-Solving Approach
Step-by-Step Breakdown: When presented with a problem or request, break it down into steps.
Clear Communication: Clearly communicate your thought process and the steps you're taking.
Task Outlining: If a task requires multiple steps or tools, outline your approach before beginning.
Transparency and Accuracy
Clarity: Always be clear about what you're doing. If you're running code, say so.
Honesty: If you're unsure about something or if a task is beyond your capabilities, communicate this clearly.
Real Results: Do not present hypothetical results as actual outcomes. Only report real results from your code execution.
Interaction Style
Concise Responses: Be concise in simple queries but provide detailed explanations for complex tasks.
Technical Language: Use technical language appropriately, but be prepared to explain concepts in simpler terms if asked.
Proactive Information: Proactively offer relevant information or alternative approaches that might be helpful.
Continuous Improvement
Clarification and Follow-Up: After completing a task, ask if the user needs any clarification or has follow-up questions.
Feedback Receptivity: Be receptive to feedback and adjust your approach accordingly.
Remember, your goal is to provide accurate, helpful, and insightful assistance by leveraging your unique capabilities in code execution and data analysis. Always strive to give the most practical and effective solution to the user's request."""

### Bedrock Agent Creation

In [9]:
print("Agent Creation in Process...")

# Create the Bedrock Agent
# This section initiates the creation of a new Bedrock Agent with the specified parameters:
# - agentName: The name of the agent, appended with a random suffix to ensure uniqueness.
# - foundationModel: The foundation model to be used by the agent (e.g., 'anthropic.claude-3-sonnet-20240229-v1:0').
# - instruction: The detailed instructions defining the agent's personality, behavior, and capabilities.
# - agentResourceRoleArn: The ARN of the IAM role assigned to the agent, granting necessary permissions.
response = bedrock_agent.create_agent(
    agentName=f"{agentName}-{randomSuffix}",
    foundationModel=foundationModel,
    instruction=instruction,
    agentResourceRoleArn=roleArn,
)

# Extract the agentId from the response for further operations.
agentId = response['agent']['agentId']

print("Checking for agent status of 'NOT_PREPARED'...")

# Wait for the agent to reach the 'NOT_PREPARED' status.
# The agent creation process involves multiple steps, and this loop ensures we proceed only when the agent reaches the 'NOT_PREPARED' status.
# This status indicates that the agent has been created but is not yet fully prepared for use.
agentStatus = ''
while agentStatus != 'NOT_PREPARED':
    # Retrieve the current status of the agent.
    response = bedrock_agent.get_agent(
        agentId=agentId
    )
    agentStatus = response['agent']['agentStatus']
    print(f"Agent status: {agentStatus}")
    time.sleep(2)  # Wait for 2 seconds before checking the status again.


Agent Creation in Process...
Checking for agent status of 'NOT_PREPARED'...
Agent status: CREATING
Agent status: NOT_PREPARED


### Enabling Code interpreter feature

In [11]:
print("Configuring code interpreter for the agent...")

# Create the agent action group
# This section configures the code interpreter action group for the agent with the specified parameters:
# - actionGroupName: The name of the action group.
# - actionGroupState: The state of the action group, set to 'ENABLED'.
# - agentId: The ID of the agent to which this action group belongs.
# - agentVersion: The version of the agent, set to 'DRAFT'.
# - parentActionGroupSignature: The predefined signature 'AMAZON.CodeInterpreter' enables the agent to generate, run, and troubleshoot code.
#   Note: The 'description', 'apiSchema', and 'actionGroupExecutor' fields must be left blank for this action group.
response = bedrock_agent.create_agent_action_group(
    actionGroupName='CodeInterpreterAction',
    actionGroupState='ENABLED',
    agentId=agentId,
    agentVersion='DRAFT',
    parentActionGroupSignature='AMAZON.CodeInterpreter'
)

# Extract the actionGroupId from the response for further operations.
actionGroupId = response['agentActionGroup']['actionGroupId']

print("Waiting for action group status of 'ENABLED'...")

# Wait for the action group to reach the 'ENABLED' status.
# This loop ensures that the action group is fully enabled before proceeding.
# The 'ENABLED' status indicates that the action group is ready for use.
actionGroupStatus = ''
while actionGroupStatus != 'ENABLED':
    # Retrieve the current status of the action group.
    response = bedrock_agent.get_agent_action_group(
        agentId=agentId,
        actionGroupId=actionGroupId,
        agentVersion='DRAFT'
    )
    actionGroupStatus = response['agentActionGroup']['actionGroupState']
    print(f"Action Group status: {actionGroupStatus}")
    time.sleep(2)  # Wait for 2 seconds before checking the status again.

print("Preparing the agent...")

# Prepare the agent for use
# This function call prepares the agent, making it fully ready for operation.
response = bedrock_agent.prepare_agent(
    agentId=agentId
)

print("Waiting for agent status of 'PREPARED'...")

# Wait for the agent to reach the 'PREPARED' status.
# This loop ensures that the agent is fully prepared before proceeding.
# The 'PREPARED' status indicates that the agent is ready for deployment.
agentStatus = ''
while agentStatus != 'PREPARED':
    # Retrieve the current status of the agent.
    response = bedrock_agent.get_agent(
        agentId=agentId
    )
    agentStatus = response['agent']['agentStatus']
    print(f"Agent status: {agentStatus}")
    time.sleep(2)  # Wait for 2 seconds before checking the status again.

print("Creating an agent alias...")

# Create an alias for the agent
# This section creates an alias for the agent with the specified parameters:
# - agentAliasName: The name of the alias.
# - agentId: The ID of the agent to which this alias belongs.
response = bedrock_agent.create_agent_alias(
    agentAliasName='test',
    agentId=agentId
)

# Extract the agentAliasId from the response for further operations.
agentAliasId = response['agentAlias']['agentAliasId']

# Wait for the agent alias to be prepared.
# This loop ensures that the agent alias is fully prepared before proceeding.
# The 'PREPARED' status indicates that the agent alias is ready for use.
agentAliasStatus = ''
while agentAliasStatus != 'PREPARED':
    # Retrieve the current status of the agent alias.
    response = bedrock_agent.get_agent_alias(
        agentId=agentId,
        agentAliasId=agentAliasId
    )
    agentAliasStatus = response['agentAlias']['agentAliasStatus']
    print(f"Agent alias status: {agentAliasStatus}")
    time.sleep(2)  # Wait for 2 seconds before checking the status again.

print('Done.\n')

# Print the final agent and alias IDs.
print(f"agentId: {agentId}, agentAliasId: {agentAliasId}")


Configuring code interpreter for the agent...
Waiting for action group status of 'ENABLED'...
Action Group status: ENABLED
Preparing the agent...
Waiting for agent status of 'PREPARED'...
Agent status: PREPARING
Agent status: PREPARED
Creating an agent alias...
Agent alias status: CREATING
Agent alias status: CREATING
Agent alias status: PREPARED
Done.

agentId: 1NZTBM0AAM, agentAliasId: IJA77PCKYX


In [ ]:
### Defining the Model_invoke funtion

In [12]:
bedrock_agent_runtime = boto3.client(service_name = 'bedrock-agent-runtime', region_name = region_name)

In [23]:
def model_invoke(inputText, showTrace=True, endSession=False):
    """
    Invokes the Bedrock Agent with the given input text, processes the response,
    and handles any exceptions that may occur during the process.

    Parameters:
    inputText (str): The prompt text to send to the agent.
    showTrace (bool): Whether to enable trace to track the agent's reasoning process (default is False).
    endSession (bool): Whether to end the session with the agent after the invocation (default is False).
    """
    try:
        # Invoke the Bedrock Agent with the provided input text.
        # This function sends a prompt to the agent and retrieves the agent's response.
        response = bedrock_agent_runtime.invoke_agent(
            agentAliasId=agentAliasId,   # [REQUIRED] The alias of the agent to use.
            agentId=agentId,             # [REQUIRED] The unique identifier of the agent to use.
            sessionId=sessionId,         # [REQUIRED] The unique identifier of the session. Use the same value across requests to continue the same conversation.
            inputText=inputText,         # The prompt text to send to the agent.
            endSession=endSession,       # Specifies whether to end the session with the agent or not.
            enableTrace=showTrace,       # Specifies whether to enable trace to track the agent's reasoning process.
        )

        # The response of this operation contains an EventStream member.
        event_stream = response["completion"]
        
        
        
         # Check if trace is enabled and process the trace information if available
        if showTrace and "trace" in response:
            trace_info = response["trace"]
            print("Trace information:")
            for trace_event in trace_info:
                print(trace_event)
                

        # Iterate through the EventStream to process the agent's response.
        for event in event_stream:
            # Each event contains a chunk, which is a part of the agent's response.
            if 'chunk' in event:
                chunk = event['chunk']
                if 'bytes' in chunk:
                    # Decode the chunk bytes to UTF-8 text and print the response part.
                    text = chunk['bytes'].decode('utf-8')
                    print(f"Model Response : {text}")
                else:
                    # Print a message if the chunk does not contain bytes.
                    print("Chunk received does not contain 'bytes' field.")
            else:
                # Print a message if the event does not contain a chunk.
                print("Event received does not contain 'chunk' field.")

    except Exception as e:
        # Print any exception that occurs during the agent invocation.
        print(f"An error occurred during agent invocation: {e}")


### Model invoke with analysis data

In [24]:

sessionId = str(uuid.uuid4())

model_invoke("""Using the car speed metrics, what is fastest car

serial_no,car,model,0_to_60_time
1,Ford,Mustang GT,4.3
2,Chevrolet,Camaro SS,4.0
3,Dodge,Challenger R/T,4.8
4,Tesla,Model S Plaid,1.99
5,BMW,M3,3.8
6,Mercedes-Benz,AMG C63,3.9
7,Audi,RS5,3.7
8,Porsche,911 Carrera,4.2
9,Lamborghini,Huracan Evo,2.9
10,Ferrari,F8 Tributo,2.9
11,McLaren,720S,2.8
12,Aston Martin,Vantage,3.5
13,Jaguar,F-Type R,3.5
14,Nissan,GT-R,2.7
15,Toyota,GR Supra,3.9

.""")

Event received does not contain 'chunk' field.
Event received does not contain 'chunk' field.
Event received does not contain 'chunk' field.
Event received does not contain 'chunk' field.
Event received does not contain 'chunk' field.
Event received does not contain 'chunk' field.
Event received does not contain 'chunk' field.
Event received does not contain 'chunk' field.
Event received does not contain 'chunk' field.
Event received does not contain 'chunk' field.
Event received does not contain 'chunk' field.
Event received does not contain 'chunk' field.
Event received does not contain 'chunk' field.
Event received does not contain 'chunk' field.
Event received does not contain 'chunk' field.
Model Response : The fastest car in the provided data is the Tesla Model S Plaid with a 0-60 mph time of 1.99 seconds.
